In [1]:
import requests
from requests import Response
youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'

res:Response = requests.get(youbike_url)

if res.status_code == 200:
    print("下載成功")
else:
    print("下載失敗")


from pydantic import BaseModel,Field,field_validator,RootModel
class Site (BaseModel):
    站名:str = Field(alias='sna') 
    行政區域:str = Field(alias='sarea')
    時間:str = Field(alias='mday')
    位置:str = Field(alias='ar')
    狀態:bool = Field(alias='act')
    總數量:int = Field(alias='total')
    可借:int = Field(alias='available_rent_bikes')
    緯度:float = Field(alias='latitude')
    經度:float = Field(alias='longitude')     
    可還:int = Field(alias='available_return_bikes')

    @field_validator('站名',mode='before')
    @classmethod
    def get_name(cls,value:str) ->str:
         return value.split('_')[-1]   

class Youbick(RootModel):
    root:list[Site]


data = Youbick.model_validate_json(res.text)
all_sites = data.model_dump()


import pandas as pd
df = pd.DataFrame(all_sites)
df

下載成功


,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
0,捷運科技大樓站,大安區,2024-06-17 19:10:30,復興南路二段235號前,True,28,5,25.02605,121.54360,23
1,復興南路二段273號前,大安區,2024-06-17 19:10:30,復興南路二段273號西側,True,21,11,25.02565,121.54357,10
2,國北教大實小東側門,大安區,2024-06-17 19:06:21,和平東路二段96巷7號,True,16,2,25.02429,121.54124,14
3,和平公園東側,大安區,2024-06-17 19:05:16,和平東路二段118巷33號,True,11,11,25.02351,121.54282,0
4,辛亥復興路口西北側,大安區,2024-06-17 19:10:28,復興南路二段368號,True,16,10,25.02153,121.54299,6
...,...,...,...,...,...,...,...,...,...,...
1418,臺大總圖書館西南側,臺大公館校區,2024-06-17 19:10:23,臺大圖書館西南側,True,30,1,25.01690,121.54031,29
1419,臺大黑森林西側,臺大公館校區,2024-06-17 18:59:20,臺大霖澤館南側,True,20,1,25.01995,121.54347,19
1420,臺大獸醫館南側,臺大公館校區,2024-06-17 18:56:20,臺大獸醫系館南側,True,24,3,25.01791,121.54242,21
1421,臺大新體育館東南側,臺大公館校區,2024-06-17 19:10:30,臺大體育館東側,True,40,9,25.02112,121.53591,31


In [ ]:
df[df['可借'] <= 3]

In [3]:
#可以得知表單的相關資訊
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1423 entries, 0 to 1422
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   站名      1423 non-null   object 
 1   行政區域    1423 non-null   object 
 2   時間      1423 non-null   object 
 3   位置      1423 non-null   object 
 4   狀態      1423 non-null   bool   
 5   總數量     1423 non-null   int64  
 6   可借      1423 non-null   int64  
 7   緯度      1423 non-null   float64
 8   經度      1423 non-null   float64
 9   可還      1423 non-null   int64  
dtypes: bool(1), float64(2), int64(3), object(4)
memory usage: 101.6+ KB


In [4]:
#query 後面一定要放str

import numpy as np
less3_df=df.query('可借<=3')
less3_df.shape

(420, 10)

In [6]:
df2=df.query('可借<=3 and 可還>=40')
df2

,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
21,臺灣科技大學正門,大安區,2024-06-17 19:06:22,基隆路四段43號(臺灣科技大學正門旁小側門靠田徑場),True,50,2,25.01451,121.54142,48
22,臺灣科技大學側門,大安區,2024-06-17 19:10:23,基隆路四段73巷與基隆路口,True,99,0,25.01295,121.53973,99
24,臺灣科技大學後門,大安區,2024-06-17 19:08:15,基隆路四段41巷68弄臺科帆船大樓旁,True,39,0,25.01182,121.54165,37
68,敦化基隆路口,大安區,2024-06-17 18:59:15,敦化南路二段218號,True,38,0,25.02215,121.54839,36
72,台北科技大學億光大樓,大安區,2024-06-17 18:58:15,忠孝東路三段81號,True,45,0,25.04177,121.53833,45
...,...,...,...,...,...,...,...,...,...,...
1391,臺大共同教室北側,臺大公館校區,2024-06-17 18:38:21,臺大共同教學館東北側,True,42,1,25.01595,121.53791,41
1399,臺大社科院西側,臺大公館校區,2024-06-17 19:09:20,臺大社科院西側,True,42,3,25.02053,121.54145,39
1400,臺大社會系館南側,臺大公館校區,2024-06-17 19:06:22,臺大社會及社工館南側,True,39,1,25.01987,121.54175,38
1405,臺大生命科學館西北側,臺大公館校區,2024-06-17 18:44:16,臺大動物博物館西側,True,30,0,25.01548,121.53825,30


In [8]:
df2=df.query('可借<=3 or 可還<=3')
df2

,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
2,國北教大實小東側門,大安區,2024-06-17 19:06:21,和平東路二段96巷7號,True,16,2,25.02429,121.54124,14
3,和平公園東側,大安區,2024-06-17 19:05:16,和平東路二段118巷33號,True,11,11,25.02351,121.54282,0
5,復興南路二段280號前,大安區,2024-06-17 19:05:23,復興南路二段280號,True,11,10,25.02429,121.54328,1
7,新生南路三段52號前,大安區,2024-06-17 19:10:30,新生南路三段52號,True,17,17,25.02112,121.53407,0
8,新生南路三段66號前,大安區,2024-06-17 18:58:15,新生南路三段66號東側,True,16,16,25.01976,121.53384,0
...,...,...,...,...,...,...,...,...,...,...
1415,臺大鄭江樓北側,臺大公館校區,2024-06-17 19:10:30,臺大新聞研究所南側,True,10,3,25.01900,121.54305,7
1416,臺大電機二館東南側,臺大公館校區,2024-06-17 19:10:30,臺大電機二館南側,True,14,0,25.01857,121.54247,14
1418,臺大總圖書館西南側,臺大公館校區,2024-06-17 19:10:23,臺大圖書館西南側,True,30,1,25.01690,121.54031,29
1419,臺大黑森林西側,臺大公館校區,2024-06-17 18:59:20,臺大霖澤館南側,True,20,1,25.01995,121.54347,19


In [11]:
df2=df.query('行政區域=="大安區" and 可還==0') #外圍為單引號,內部的字串需要為雙銀號
df2

,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
3,和平公園東側,大安區,2024-06-17 19:05:16,和平東路二段118巷33號,True,11,11,25.02351,121.54282,0
7,新生南路三段52號前,大安區,2024-06-17 19:10:30,新生南路三段52號,True,17,17,25.02112,121.53407,0
8,新生南路三段66號前,大安區,2024-06-17 18:58:15,新生南路三段66號東側,True,16,16,25.01976,121.53384,0
9,新生南路三段82號前,大安區,2024-06-17 19:06:16,新生南路三段82號,True,20,20,25.01894,121.53361,0
11,和平復興路口西北側,大安區,2024-06-17 19:07:15,復興南路二段236號,True,9,9,25.02543,121.54332,0
15,溫州公園,大安區,2024-06-17 19:07:15,羅斯福路三段283巷11號,True,5,5,25.01895,121.53156,0
37,臺大綜合體育館停車場前,大安區,2024-06-17 18:52:15,臺大綜合體育館北側,True,25,25,25.02220,121.53626,0
46,臥龍公園,大安區,2024-06-17 18:39:15,辛亥路三段223號西側,True,14,14,25.01623,121.55037,0
82,龍門廣場,大安區,2024-06-17 18:49:15,敦化南路一段與敦化南路一段236巷口,True,70,70,25.04092,121.54825,0
103,德安公園(四維路66巷),大安區,2024-06-17 19:09:14,四維路66巷6號北側,True,14,13,25.03473,121.54767,0


In [13]:
n=0
df2=df.query('行政區域=="大安區" and 可借==@n') #建立一個變數，讓資料更靈活
df2

,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
17,溫州停車場,大安區,2024-06-03 14:08:52,新生南路三段54巷/溫州街口,False,5,0,25.02093,121.53310,5
22,臺灣科技大學側門,大安區,2024-06-17 19:10:23,基隆路四段73巷與基隆路口,True,99,0,25.01295,121.53973,99
24,臺灣科技大學後門,大安區,2024-06-17 19:08:15,基隆路四段41巷68弄臺科帆船大樓旁,True,39,0,25.01182,121.54165,37
26,臺大環研大樓,大安區,2024-06-17 18:50:24,芳蘭路/基隆路三段155巷口,True,15,0,25.01234,121.54503,15
35,臺大社科院圖書館前,大安區,2024-06-17 19:06:16,辛亥路與復興南路口西南側,True,27,0,25.02101,121.54284,27
55,和平泰順街口,大安區,2024-06-17 19:10:30,和平東路一段/泰順街口東南側,True,9,0,25.02621,121.53171,9
67,敦化南路二段293巷口南側,大安區,2024-06-17 18:58:15,敦化南路二段293巷口南側,True,14,0,25.02370,121.54899,14
68,敦化基隆路口,大安區,2024-06-17 18:59:15,敦化南路二段218號,True,38,0,25.02215,121.54839,36
72,台北科技大學億光大樓,大安區,2024-06-17 18:58:15,忠孝東路三段81號,True,45,0,25.04177,121.53833,45
76,復興南路1段340巷口,大安區,2024-06-17 19:09:14,復興南路一段信義路三段147巷45弄口,True,50,0,25.03701,121.54348,50
